In [2]:
import pandas as pd
import numpy as np

In [4]:
df = pd.read_csv('/Users/data/VsCodeProjects/DataScience/data/raw/XAU_1d_data.csv', sep=';')

In [5]:
df.head(5)

,Date,Open,High,Low,Close,Volume
0,2004.06.11 00:00,384.0,384.8,382.8,384.1,272
1,2004.06.14 00:00,384.3,385.8,381.8,382.8,1902
2,2004.06.15 00:00,382.8,388.8,381.1,388.6,1951
3,2004.06.16 00:00,387.1,389.8,382.6,383.8,2014
4,2004.06.17 00:00,383.6,389.3,383.0,387.6,1568


In [6]:
df['Date'] = pd.to_datetime(df['Date'])

In [7]:
df.describe()

,Date,Open,High,Low,Close,Volume
count,5391,5391.000000,5391.000000,5391.000000,5391.000000,5.391000e+03
mean,2014-12-21 05:40:02.003339008,1351.249024,1361.248208,1341.127752,1351.567527,8.707645e+04
min,2004-06-11 00:00:00,382.800000,384.800000,381.100000,382.800000,2.000000e+00
25%,2009-09-26 12:00:00,1008.135000,1018.045000,999.275000,1007.215000,2.487950e+04
50%,2014-12-24 00:00:00,1292.420000,1299.500000,1285.410000,1292.480000,7.173500e+04
75%,2020-03-17 12:00:00,1723.255000,1735.060000,1708.605000,1724.460000,1.153610e+05
max,2025-06-06 00:00:00,3435.210000,3499.940000,3366.730000,3431.480000,1.719636e+06
std,NaN,548.743404,552.815301,545.007579,549.292651,1.073524e+05


In [8]:
df = df.sort_values('Date').reset_index(drop=True)

df['log_return_1d'] = np.log(df['Close'].shift(-1) / df['Close'])
df['direction'] = (df['log_return_1d'] > 0).astype(int)
df = df.iloc[:-1]

In [15]:
df

,Date,Open,High,Low,Close,Volume,log_return_1d,direction
0,2004-06-11,384.00,384.80,382.80,384.10,272,-0.003390,0
1,2004-06-14,384.30,385.80,381.80,382.80,1902,0.015038,1
2,2004-06-15,382.80,388.80,381.10,388.60,1951,-0.012429,0
3,2004-06-16,387.10,389.80,382.60,383.80,2014,0.009852,1
4,2004-06-17,383.60,389.30,383.00,387.60,1568,0.017138,1
...,...,...,...,...,...,...,...,...
5384,2025-05-29,3286.86,3331.29,3245.43,3317.89,539454,-0.008767,0
5385,2025-05-30,3318.68,3322.72,3271.47,3288.93,496062,0.027503,1
5386,2025-06-02,3298.76,3382.86,3295.75,3380.64,461738,-0.008236,0
5387,2025-06-03,3383.51,3392.19,3333.08,3352.91,436080,0.005752,1


In [9]:
df['body'] = df['Close'] - df['Open']
df['range'] = df['High'] - df['Low']
df['upper_shadow'] = df['High'] - df[['Open', 'Close']].max(axis=1)
df['lower_shadow'] = df[['Open', 'Close']].min(axis=1) - df['Low']


In [10]:
df['body_pct'] = df['body'] / df['Close']
df['range_pct'] = df['range'] / df['Close']

In [11]:
for w in [1, 3, 5, 10, 20]:
    df[f'log_return_{w}'] = np.log(df['Close'] / df['Close'].shift(w))
    

In [12]:
df['true_range'] = np.maximum(
    df['High'] - df['Low'],
    np.maximum(
        abs(df['High'] - df['Close'].shift(-1)),
        abs(df['Low'] - df['Close'].shift(-1))
    )
    )

for w in [5, 10, 20]:
    df[f'atr_{w}'] = df['true_range'].rolling(w).mean()
    df[f'volatility_{w}'] = df['log_return_1'].rolling(w).std()

In [13]:
for w in [5, 10, 20, 50]:
    sma = df['Close'].rolling(w).mean()
    df[f'sma_dist_{w}'] = (df['Close'] - sma) / df['Close']

In [14]:
df["volume_change"] = df["Volume"].pct_change()

for w in [5, 20]:
    df[f"volume_z_{w}"] = (
        (df["Volume"] - df["Volume"].rolling(w).mean()) /
        df["Volume"].rolling(w).std()
    )


In [15]:
df["day_of_week"] = df["Date"].dt.weekday
df["week_of_year"] = df["Date"].dt.isocalendar().week.astype(int)
df["month"] = df["Date"].dt.month
df["is_month_end"] = df["Date"].dt.is_month_end.astype(int)


In [16]:
feature_cols = [c for c in df.columns if c not in [
    "Date", "Open", "High", "Low", "Close", "Volume",
    "log_return_1d", "direction"
]]

df[feature_cols] = df[feature_cols].shift(1)


In [17]:
df = df.dropna().reset_index(drop=True)


In [18]:
train = df.iloc[:int(len(df)*0.7)]
test  = df.iloc[int(len(df)*0.7):]


In [19]:
target = "log_return_1d"

X_train = train[feature_cols]
y_train = train[target]

X_test  = test[feature_cols]
y_test  = test[target]


In [21]:
import lightgbm as lgb

model = lgb.LGBMRegressor(
    n_estimators=500,
    learning_rate=0.01,
    max_depth=5,
    num_leaves=32,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42
)

model.fit(
    X_train, y_train,
    eval_set=[(X_test, y_test)],
    eval_metric="l2",
    verbose=50
)



ModuleNotFoundError: No module named 'lightgbm'